In [1]:
#@title import libraries
import torch
import math
import numpy as np
# import wandb
import meent
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy
from datetime import datetime
import time
# from google.colab import drive
# drive.mount('/content/drive')
start = time.time()

P = 1000
t = 1500
h = 500
N_L = 4
N_C = 32
pixel_min_width = P/N_C
pixel_min_height = t/N_L


In [2]:
n_air = 1.0

backend = 2  # torch
device = 0
pol = 1 # 0: TE, 1: TM


theta = 0 * torch.pi / 180  # angle of incidence
phi = 0 * torch.pi / 180  # angle of rotation
period = torch.tensor([P])  # length of the unit cell. Here it's 1D.

fourier_order = [60]
type_complex = torch.complex128
#grating_type = 0  # grating type: 0 for 1D grating without rotation (phi == 0)

#@title optimization
res_x = N_C*2
res_z = 15

n1 = 1.5
n2 = 2.0
# n1 = 1.0
# n2 = 1.0

n_I = 1.0  # n_incidence
n_II = n2  # n_transmission,


thickness = torch.tensor([500.] + [pixel_min_height for layer in range(N_L)] + [50. for i in range(10)] + [100000.])  # thickness of each layer, from top to bottom.

# functions
def return_device(patterning):
    Layer_air =  n_air * torch.ones(1,1,N_C)
    Layer_patterning = torch.from_numpy(np.expand_dims(patterning, axis=1))
    Layer_detector =  n2 * torch.ones(12,1,N_C)

    ucell = torch.cat((Layer_air,Layer_patterning,Layer_detector))
    return ucell

def F(field_cell_meent_1d):

    R_index_i = 0
    R_index_f = 16
    
    G_index_i = 16
    G_index_f = 32

    B_index_i = 32
    B_index_f = 48

    I_index_i = 48
    I_index_f = 64
    
    B = torch.sum(torch.abs(field_cell_meent_1d[B_index_i:B_index_f])**2)
    G1 = torch.sum(torch.abs(field_cell_meent_1d[G_index_i:G_index_f])**2)
    R = torch.sum(torch.abs(field_cell_meent_1d[R_index_i:R_index_f])**2)
    G2 = torch.sum(torch.abs(field_cell_meent_1d[I_index_i:I_index_f])**2)

    G = G1 + G2

    return R, G, B

# def shift_first_dimension(field_cell_meent_2d):
#     return field_cell_meent_2d[1:] + field_cell_meent_2d[:1]

def data_augmentation(patterning, field_cell_meent_color):

    patterning_aug_list, field_cell_aug_list = [], []
    patterning_aug_list.append(patterning)
    field_cell_aug_list.append(field_cell_meent_color)

    for i in range(N_C-1):
        patterning = np.roll(patterning, -1, axis = 1)
        patterning_aug_list.append(patterning)  
        # print(patterning)
        field_cell_meent_color = np.roll(field_cell_meent_color, -1, axis = 1)
        # print(field_cell_meent_color)
        field_cell_aug_list.append(field_cell_meent_color)

    return patterning_aug_list, field_cell_aug_list



wav_len_step = 10
wav_len_list = [400 + i * wav_len_step for i in range((300) // wav_len_step + 1)]

Tot_F_list = []
oe_list = []
binary_lev_list = []
result_folder_path = r'D:\Sunghyun Nam\SANZABOO\data_Ez'
os.makedirs(result_folder_path, exist_ok=True)
input_patterning_folder = os.path.join(result_folder_path, 'patternings')
output_result_folder = os.path.join(result_folder_path, 'true_results')
os.makedirs(input_patterning_folder, exist_ok=True)
os.makedirs(output_result_folder, exist_ok=True)

In [17]:
num_samples = 10000

for epoch in tqdm(range(num_samples)) :
    
    random_array = np.ones((N_L, N_C)) *n1 + (n2-n1)* np.random.randint(2, size=(N_L, N_C))
    # print(random_array)
    resulting = []
    ucells = []
    #def closure():
    Tot_F = 0
    epoch_oe_list = []
    epoch_fom_list = []
    T_list = []
    
    field_cell_meent_color = []
    field_cell_meent_Ex_color = []

    for color in range(len(wav_len_list)):
        # print(color)
        wav_len = wav_len_list[color]
        # print(wav_len)
        # patterning = torch.sigmoid((1+0.01*epoch)*latent) # binarization, g(t)

        ucell = return_device(random_array)
        ucells.append(ucell.tolist())
        #mee = meent.call_mee(backend=backend, grating_type=grating_type, pol=pol, n_I=n_I, n_II=n_II, theta=theta, phi=phi, fourier_order=fourier_order, wavelength=wav_len, period=period, ucell=ucell, thickness=thickness, type_complex=type_complex, device=device, fft_type=0, improve_dft=True)
        mee = meent.call_mee(backend=backend, pol=pol, n_top=n_I, n_bot=n_II, theta=theta, phi=phi, fto=fourier_order, wavelength=wav_len, period=period, thickness=thickness, type_complex=type_complex, device=device, fourier_type=0) # After update

        mee.ucell = ucell

        de_ri, de_ti = mee.conv_solve()
        T_list.append(de_ti.sum())

        field_cell_meent = mee.calculate_field(res_x=res_x,res_z=res_z)
        observation = res_z * (1+ N_L + 10)
        field_cell_meent_Ex = field_cell_meent[:observation, 0, :, 0]
        field_cell_meent_1d = field_cell_meent[observation, 0, :, 0]
        

        field_cell_meent_color.append(field_cell_meent_1d.numpy())
        field_cell_meent_Ex_color.append(field_cell_meent_Ex.numpy())
        #print(f"Shape of field_cell_meent_1d for color {color}: {field_cell_meent_1d.shape}")
        
    #print(f"Shape of field_cell_meent_color: {len(field_cell_meent_color)}")
    field_cell_meent_Ex_color = np.array(field_cell_meent_Ex_color)
    
    name = str(epoch).zfill(8)
    patterning_path = os.path.join(input_patterning_folder, 'patterning_'+name+'.npy')
    result_path = os.path.join(output_result_folder, 'result_Ex_'+name+'.npy')
    np.save(patterning_path, random_array)
    np.save(result_path, field_cell_meent_Ex_color)

    print(f"{epoch} : simulation done")
    

end_time = time.time()

# 걸린 시간 계산
elapsed_time = end_time - start
print(elapsed_time)


  0%|          | 1/10000 [00:27<75:01:49, 27.01s/it]

0 : simulation done


  0%|          | 2/10000 [00:53<73:47:00, 26.57s/it]

1 : simulation done


  0%|          | 3/10000 [01:19<73:26:49, 26.45s/it]

...

2375 : simulation done


 24%|██▍       | 2377/10000 [17:26:23<58:06:34, 27.44s/it]

2376 : simulation done


 24%|██▍       | 2378/10000 [17:26:50<57:54:40, 27.35s/it]

2377 : simulation done
